In [1]:
import json
import random
from pathlib import Path

input_path = Path("/Users/sheillaschool/Documents/final/Thesis_PredictingNewsOutdatedness_Logistic_Decay/model/labelling/data/manual/rulebased_manual.json")
output_path = input_path.parent / "manual_review_sample.json"

with open(input_path, "r") as f:
    data = json.load(f)

manual_label_1 = []
manual_label_0 = []

for key, article in data.items():
    if article.get("label_comment") == "manual":
        label = article.get("label")
        if label == 1:
            manual_label_1.append((key, article))
        elif label == 0:
            manual_label_0.append((key, article))

random.seed(42)
sample_1 = random.sample(manual_label_1, min(60, len(manual_label_1)))
sample_0 = random.sample(manual_label_0, min(60, len(manual_label_0)))

combined = sample_1 + sample_0
random.shuffle(combined)

final_sample = {}
for key, article in combined:
    article_copy = article.copy()
    article_copy["reviewed_label"] = None
    final_sample[key] = article_copy


with open(output_path, "w") as f:
    json.dump(final_sample, f, indent=2)

print(f"✅ Saved {len(final_sample)} articles to {output_path}")


✅ Saved 120 articles to /Users/sheillaschool/Documents/final/Thesis_PredictingNewsOutdatedness_Logistic_Decay/model/labelling/data/manual/manual_review_sample.json


In [2]:
import json
from sklearn.metrics import cohen_kappa_score, accuracy_score

# Path to your labeled JSON file
json_path = "/Users/sheillaschool/Documents/final/Thesis_PredictingNewsOutdatedness_Logistic_Decay/model/labelling/data/manual/manual_review_sample_filled.json"  # 🔁 Change this to your actual path

# Load the JSON data
with open(json_path, "r") as f:
    data = json.load(f)

# Collect labels where both annotators provided values
manual_labels = []
reviewed_labels = []
disagreements = []

for article_id, article in data.items():
    label = article.get("label")
    reviewed_label = article.get("reviewed_label")

    # Only compare when both labels are present
    if label is not None and reviewed_label is not None:
        manual_labels.append(label)
        reviewed_labels.append(reviewed_label)
        if label != reviewed_label:
            disagreements.append((article_id, label, reviewed_label, article.get("title")))

# Calculate agreement metrics
kappa = cohen_kappa_score(manual_labels, reviewed_labels)
simple_agreement = accuracy_score(manual_labels, reviewed_labels)

print(f"🔍 Inter-Annotator Agreement Results:")
print(f"🟦 Cohen's Kappa: {kappa:.3f}")
print(f"🟩 Simple Agreement: {simple_agreement:.3f}")
print(f"🔻 Total samples compared: {len(manual_labels)}")
print(f"❗ Disagreements: {len(disagreements)}")

# Print sample disagreements
if disagreements:
    print("\nSample Disagreements:")
    for i, (article_id, label1, label2, title) in enumerate(disagreements[:5]):
        print(f"{i+1}. ID: {article_id}, Manual: {label1}, Reviewer: {label2}, Title: {title}")


🔍 Inter-Annotator Agreement Results:
🟦 Cohen's Kappa: 0.700
🟩 Simple Agreement: 0.850
🔻 Total samples compared: 120
❗ Disagreements: 18

Sample Disagreements:
1. ID: 916, Manual: 1, Reviewer: 0, Title: NFL scraps 'End Racism' message from Super Bowl LIX end zones
2. ID: 10237, Manual: 1, Reviewer: 0, Title: Pharrell Williams and Michel Gondry Scrap Universal Musical ‘Golden’ in Post-Production
3. ID: 5824, Manual: 0, Reviewer: 1, Title: Man who exploded Tesla Cybertruck outside Trump hotel in Las Vegas used AI, Police says – Ya Libnan
4. ID: 10496, Manual: 0, Reviewer: 1, Title: Costa Rica’s Gordo Navideño Is A Christmas Tradition
5. ID: 5495, Manual: 1, Reviewer: 0, Title: Mavericks vs. Lakers tickets: Cheapest prices, best seats to see Luka Doncic return to Dallas vs. Anthony Davis
